# Big Red Data Challenge (Spring 2025)

In [18]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import data
donor_df = pd.read_excel("./data/all_transactions.xlsx")
hospital_df = pd.read_excel("./data/survey_responses.xlsx")

## Donor Data

Potential Data Goals:
- Improved Fundraising Strategies: Donor data helps identifying trends, preferred donation methods, and the best time to solicit gifts
- Enhanced Donor Retention and Engagement: Help us understand donor behaviors, such as frequency and amount of giving
- Better Decision-Making: Help us prioritize high-value donors, target new donor prospects

### 1. Clean & Explore Data

In [5]:
donor_df.head()

,Account Number,Date,Revenue Amount,Type,Fund,Campaign,Appeal,Primary City,Primary State,Primary ZIP Code
0,8231,2025-02-18,20.76,Recurring Donation Payment,Unrestricted,NaN,NaN,Columbus,OH,43214-4014
1,2139,2025-02-17,78.30,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,NaN,Galena,OH,43021
2,1566,2025-02-15,52.30,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,NaN,Darien,CT,06820-3204
3,10382,2025-02-15,21.11,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,NaN,Columbus,OH,43214
4,9739,2025-02-15,24.23,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,BTJ Website,Dublin,OH,43017


In [27]:
# Remove wrong data points
donor_df = donor_df.loc[donor_df['Date'] > '2014-12-31']
donor_df['Primary City'] = donor_df['Primary City'].str.title()
donor_df.loc[donor_df['Primary City'].str.contains('james', case=False, na=False)]


,Account Number,Date,Revenue Amount,Type,Fund,Campaign,Appeal,Primary City,Primary State,Primary ZIP Code
15,32,2025-02-11,100.00,Recurring Donation Payment,Unrestricted,NaN,NaN,St. James City,FL,33956
23,9719,2025-02-06,51.80,Donation,Golisano Children's Hospital,2025 Celebrating A Decade of Hope and Healing,NaN,St James City,FL,33956
39,9722,2025-01-29,26.31,Donation,Unrestricted,2025 Celebrating A Decade of Hope and Healing,NaN,Saint James City,FL,33956
40,825,2025-01-29,52.30,Recurring Donation Payment,Unrestricted,2025 Celebrating A Decade of Hope and Healing,NaN,Saint James City,FL,33956
82,32,2025-01-11,100.00,Recurring Donation Payment,Unrestricted,NaN,NaN,St. James City,FL,33956
...,...,...,...,...,...,...,...,...,...,...
9187,1041,2017-06-12,2500.00,Donation,Unrestricted,2017 5k,NaN,St. James City,FL,33956
9198,1402,2017-06-09,250.00,Donation,Unrestricted,2017 5k,NaN,Saint James City,FL,33956-2118
9307,1041,2016-10-20,2500.00,Donation,Unrestricted,NaN,NaN,St. James City,FL,33956
9400,32,2016-03-15,1800.00,Donation,Unrestricted,NaN,NaN,St. James City,FL,33956


In [34]:
donor_df.dtypes

Account Number               int64
Date                datetime64[ns]
Revenue Amount             float64
Type                        object
Fund                        object
Campaign                    object
Appeal                      object
Primary City                object
Primary State               object
Primary ZIP Code            object
dtype: object

In [45]:
last_date = max(donor_df['Date'])
donor_df["Recent Date"] = last_date - donor_df["Date"]
donor_df["id"] = donor_df["Account Number"]

In [42]:
donor_df

,Account Number,Date,Revenue Amount,Type,Fund,Campaign,Appeal,Primary City,Primary State,Primary ZIP Code,Recent Date
0,8231,2025-02-18,20.76,Recurring Donation Payment,Unrestricted,NaN,NaN,Columbus,OH,43214-4014,0 days
1,2139,2025-02-17,78.30,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,NaN,Galena,OH,43021,1 days
2,1566,2025-02-15,52.30,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,NaN,Darien,CT,06820-3204,3 days
3,10382,2025-02-15,21.11,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,NaN,Columbus,OH,43214,3 days
4,9739,2025-02-15,24.23,Recurring Donation Payment,Unrestricted,Sam's Fans 2024,BTJ Website,Dublin,OH,43017,3 days
...,...,...,...,...,...,...,...,...,...,...,...
9482,296,2015-01-01,100.00,Donation,Unrestricted,NaN,NaN,Columbus,OH,43220-4325,3701 days
9483,163,2015-01-01,50.00,Donation,Unrestricted,NaN,NaN,Powell,OH,43065,3701 days
9484,147,2015-01-01,500.00,Donation,Unrestricted,NaN,NaN,NaN,NaN,NaN,3701 days
9485,8035,2015-01-01,100.00,Donation,Unrestricted,NaN,NaN,Columbus,OH,43221,3701 days


In [49]:
donor_frequency = donor_df.groupby("id", as_index = False).agg({"Revenue Amount":"sum", 
                                        "Recent Date":"min",
                                        "Account Number":"count"})
donor_frequency = donor_frequency.rename(columns={"Revenue Amount":"Monetary",
                                                  "Recent Date":"Recency",
                                                  "Account Number":"Frequency"})
donor_frequency

,id,Monetary,Recency,Frequency
0,4,1475.00,116 days,5
1,5,9865.07,17 days,96
2,6,450.00,2308 days,3
3,8,2225.58,694 days,7
4,11,4045.60,845 days,5
...,...,...,...,...
2610,10823,200.00,49 days,1
2611,10824,1000.00,35 days,1
2612,10825,26.31,23 days,1
2613,10826,0.00,23 days,1


## Hospital Data

Potential Data Goals:
- Resource Allocation: Help us allocate resource effectively

In [6]:
hospital_df.head()

,#,Reporting Quarter,Hospital Name,Art therapy,Music therapy,Artist in Residence,Addressing communication barriers,Addressing feelings,Coping with hospitalization,Decreasing pain perception,...,Number of White or Caucasian patients,Number of Black or African American patients,Number of patients of another race or unknown race,Number of Hispanic or Latino patients,Response Type,Start Date (UTC),Stage Date (UTC),Submit Date (UTC),Network ID,Tags
0,b5fe0pz2mibn56085b5fe4abz1x06b77,January - March 2024,Akron Children's Hospital,Art therapy,Music therapy,NaN,Addressing communication barriers,Addressing feelings,Coping with hospitalization,Decreasing pain perception,...,408.0,111.0,30.0,24.0,completed,2025-04-02 14:15:49,NaN,2025-04-02 14:23:31,72e438beab,NaN
1,86qiuhhqowuw8xcfzhlua86qiuhis9nu,April - June 2024,Akron Children's Hospital,Art therapy,Music therapy,NaN,Addressing communication barriers,Addressing feelings,Coping with hospitalization,Decreasing pain perception,...,398.0,90.0,32.0,19.0,completed,2025-04-02 14:02:24,NaN,2025-04-02 14:15:01,72e438beab,NaN
2,v1wtywii1ao2d4ynuv1v1wt9y0r26n27,October - December 2024,Nationwide Children's Hospital,NaN,Music therapy,NaN,Addressing communication barriers,Addressing feelings,NaN,Decreasing pain perception,...,56.0,14.0,3.0,4.0,completed,2025-02-10 13:32:47,NaN,2025-02-10 13:43:50,dcc9944acb,NaN
3,mcztrdruc1khh47or7mcztrrv9vfcpvd,October - December 2024,University Hospitals- Rainbow Babies and Child...,Art therapy,NaN,NaN,NaN,NaN,NaN,NaN,...,65.0,41.0,0.0,2.0,completed,2025-01-21 22:59:06,NaN,2025-01-21 23:12:09,656147dd83,NaN
4,jzy2f2pv8gxahz0djzy2o3y6cj0vm66h,October - December 2024,ProMedica Russell J. Ebeid Children's Hospital,NaN,Music therapy,NaN,Addressing communication barriers,Addressing feelings,Coping with hospitalization,Decreasing pain perception,...,52.0,20.0,3.0,4.0,completed,2025-01-17 20:20:59,NaN,2025-01-17 20:42:31,8c7ae757df,NaN
